In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

disable_eager_execution()

tf.__version__

'1.15.0'

In [0]:
import torch
torch.cuda.current_device()
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My\ Drive/WPI/DL/


In [0]:
# %ls

img_height = 48
img_width = 48
batch_size = 64

x_train = np.load('faces.npy')
y_train_1 = np.load('ages.npy')


y_train_1 = np.ceil(y_train_1)
# y_train = y_train_1
print(y_train_1.shape)
y_train = np.zeros([y_train_1.shape[0],10])
print(y_train.shape)

for i in range(y_train_1.shape[0]):
  # print(type(y_train_1[i]))
  y_train[i][int((y_train_1[i]-1)/10)] = 1 
  # print(y_train)

# Split the data

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, shuffle= True)


print(x_train.shape)

x_train = np.stack((x_train,x_train,x_train),3)
x_test = np.stack((x_test,x_test,x_test),3)

print(x_train.shape)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255



(7500,)
(7500, 10)
(6750, 48, 48)
(6750, 48, 48, 3)


In [0]:
def mean_squared_err(y_true, y_pred):
  output = (K.argmax(y_pred, axis=1)*10)+5
  y_actual = (K.argmax(y_true, axis=1)*10)+5
  score = K.mean(K.square(output-y_actual))
  return score

In [0]:
# create the base pre-trained model
# base_model = VGG16(weights='imagenet', include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False


In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=[mean_squared_err])

In [0]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)



model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=2000,
         validation_split=0.1,
         shuffle = True,
          callbacks=[checkpointer])

In [0]:
!kill -9 -1

In [0]:
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=20,
         validation_split=0.1,
         shuffle = True)

In [0]:

score = model.evaluate(x_test, y_test, verbose=0)

print('\n', 'Test accuracy:', score[1])



 Test accuracy: 178.375
